In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,922 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,556 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main 

In [ ]:
import os
import sys
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

In [ ]:
# Context

#Creating spark context-Its like connecting to spark cluster
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import requests
from pyspark.sql import SparkSession

# Asegúrate de tener una sesión de Spark
spark = SparkSession.builder.getOrCreate()

players_match_csv = '/content/drive/MyDrive/Players_Match-2_FINAL.csv'
#players_team_csv = '/content/drive/MyDrive/UNIVERSIDAD/SEMESTRE05/PROCESAMIENTO-DATOS/PARCIAL/DATA LIMPIA/Players_Team.csv'
#teams_match_res_csv = '/content/drive/MyDrive/UNIVERSIDAD/SEMESTRE05/PROCESAMIENTO-DATOS/PARCIAL/DATA LIMPIA/Equipos_Match_Fusion_Results.csv'

# Lee el archivo CSV en un DataFrame de Spark
players_match = spark.read.csv(players_match_csv, header=True, inferSchema=True)
#players_team = spark.read.csv(players_team_csv, header=True, inferSchema=True)
#teams_res = spark.read.csv(teams_match_res_csv, header=True, inferSchema=True)

players_match.printSchema()
#players_team.printSchema()
#teams_res.printSchema()


root
 |-- game_id: integer (nullable = true)
 |-- team_id: integer (nullable = true)
 |-- team_name: string (nullable = true)
 |-- date: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_id: integer (nullable = true)
 |-- player_position_info: string (nullable = true)
 |-- player_rating: double (nullable = true)
 |-- good_high_claim: integer (nullable = true)
 |-- touches: integer (nullable = true)
 |-- saves: integer (nullable = true)
 |-- total_pass: integer (nullable = true)
 |-- formation_place: integer (nullable = true)
 |-- accurate_pass: integer (nullable = true)
 |-- aerial_won: integer (nullable = true)
 |-- aerial_lost: integer (nullable = true)
 |-- fouls: integer (nullable = true)
 |-- total_scoring_att: integer (nullable = true)
 |-- total_tackle: integer (nullable = true)
 |-- won_contest: integer (nullable = true)
 |-- penalty_conceded: integer (nullable = true)
 |-- blocked_scoring_att: integer (nullable = true)
 |-- man_of_the_match: integ

In [ ]:
columns_to_display = ["goals", "goal_assist", "total_scoring_att", "att_pen_goal",
                      "won_contest", "fouls", "total_tackle", "blocked_scoring_att",
                      "aerial_won", "saves", "penalty_save", "good_high_claim",
                      "error_lead_to_goal", "man_of_the_match", "clearance_off_line",
                      "accurate_pass"]

# Seleccionar las columnas del DataFrame
selected_columns_df = players_match.select(columns_to_display)

# Mostrar las columnas seleccionadas
selected_columns_df.show()

+-----+-----------+-----------------+------------+-----------+-----+------------+-------------------+----------+-----+------------+---------------+------------------+----------------+------------------+-------------+
|goals|goal_assist|total_scoring_att|att_pen_goal|won_contest|fouls|total_tackle|blocked_scoring_att|aerial_won|saves|penalty_save|good_high_claim|error_lead_to_goal|man_of_the_match|clearance_off_line|accurate_pass|
+-----+-----------+-----------------+------------+-----------+-----+------------+-------------------+----------+-----+------------+---------------+------------------+----------------+------------------+-------------+
|    0|          1|                0|           0|          0|    2|           0|                  0|         0|    0|           0|              0|                 0|               0|                 0|           21|
|    0|          0|                2|           0|          0|    1|           0|                  1|         2|    0|           0| 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


null_por = spark.sql(
    """
    SELECT
      player_position_info, saves, penalty_save, good_high_claim, error_lead_to_goal, position_score,normalized_position_score
    FROM players_data
    WHERE player_position_info = "POR"
    """
)

# Aplicación de las formulas

In [ ]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType

def performance_score(position, goals, goal_assist, total_scoring_att, att_pen_goal,
                      won_contest, fouls, total_tackle, blocked_scoring_att, aerial_won,
                      saves, penalty_save, good_high_claim, error_lead_to_goal, clearance_off_line,
                      man_of_the_match, accurate_pass, last_man_tackle,total_pass):
    if position == 'DC':  # Delantero Centro
        performance = (goals * 2.38 + goal_assist * 0.95 +
                       total_scoring_att * 1.19 + att_pen_goal * 1.43 +
                       won_contest * 0.48 - fouls * 0.24 +
                       man_of_the_match * 3.33)
    elif position == 'DFC':  # Defensa Central
       performance = (total_tackle * 1.5 + aerial_won * 2.0 + clearance_off_line * 1.0 +blocked_scoring_att * 1.7 + man_of_the_match * 2.0 + last_man_tackle * 1.5 + accurate_pass * 0.3)
    elif position in ['LI', 'LD']:  # Lateral Izquierdo / Derecho
        performance = (total_tackle * 1.04 + aerial_won * 1.04 + won_contest * 2.08 - fouls * 0.63 + accurate_pass * 1.46 + man_of_the_match * 3.75)
    elif position == 'POR':  # Portero
         performance = (saves * 3 + penalty_save * 1.5 + good_high_claim * 1.5 - error_lead_to_goal * 3.5 + accurate_pass * 0.5)
    elif position in ['MC']:  # Centrocampista Medio / Mediocampista Central
        performance = (accurate_pass * 1.0 + total_tackle * 1.2 + won_contest * 1 + goal_assist * 2.0 + goals * 2.2 + man_of_the_match * 2 + total_pass * 0.7 + aerial_won * 1.3)
    elif position in ['EI', 'ED']:  # Extremo Izquierdo / Derecho
        performance = (goals * 2.5 + goal_assist * 1.25 + total_scoring_att * 1.04 + accurate_pass * 0.65 + won_contest * 1.65 + man_of_the_match * 2.92)
    else:
        performance = 0  # Default return if no position matches
    return performance

# Convertir la función Python a UDF de Spark
udf_performance_score = udf(performance_score, FloatType())

# Aplicar la UDF a los datos y añadir una nueva columna con el resultado
players_match = players_match.withColumn("position_score", udf_performance_score(col("player_position_info"),
                                                                                 col("goals"),
                                                                                 col("goal_assist"),
                                                                                 col("total_scoring_att"),
                                                                                 col("att_pen_goal"),
                                                                                 col("won_contest"),
                                                                                 col("fouls"),
                                                                                 col("total_tackle"),
                                                                                 col("blocked_scoring_att"),
                                                                                 col("aerial_won"),
                                                                                 col("saves"),
                                                                                 col("penalty_save"),
                                                                                 col("good_high_claim"),
                                                                                 col("error_lead_to_goal"),
                                                                                 col("clearance_off_line"),
                                                                                 col("man_of_the_match"),
                                                                                 col("accurate_pass"),
                                                                                 col("last_man_tackle"),
                                                                                 col("total_pass")
                                                                                 ))

players_match.show(500)

+-------+-------+--------------------+----------+--------------------+---------+--------------------+-------------+---------------+-------+-----+----------+---------------+-------------+----------+-----------+-----+-----------------+------------+-----------+----------------+-------------------+----------------+-----+------------+-----------+--------------+-----------+------------------+----------------+-------------+--------+------------------+---------+---------------+--------------+------------+------------+------------+--------------+
|game_id|team_id|           team_name|      date|         player_name|player_id|player_position_info|player_rating|good_high_claim|touches|saves|total_pass|formation_place|accurate_pass|aerial_won|aerial_lost|fouls|total_scoring_att|total_tackle|won_contest|penalty_conceded|blocked_scoring_att|man_of_the_match|goals|att_pen_goal|goal_assist|six_yard_block|yellow_card|error_lead_to_goal|post_scoring_att|second_yellow|red_card|clearance_off_line|own_goal

In [ ]:
from pyspark.sql.window import Window
import pyspark.sql.functions as F

# Calcular el mínimo y máximo 'position_score' para cada 'player_position_info'
windowSpec  = Window.partitionBy('player_position_info')

players_match = players_match.withColumn('min_score', F.min('position_score').over(windowSpec))
players_match = players_match.withColumn('max_score', F.max('position_score').over(windowSpec))
players_match = players_match.withColumn('min_rating', F.min('player_rating').over(windowSpec))
players_match = players_match.withColumn('max_rating', F.max('player_rating').over(windowSpec))

def normalize_score(score, min_score, max_score):
    # Si max_score y min_score son iguales, la normalización no es posible; retorna None o algún valor por defecto
    if max_score == min_score:
        return None
    else:
        return (score - min_score) / (max_score - min_score)

def normalize_rating(player_rating, min_rating, max_rating):
    # Si max_score y min_score son iguales, la normalización no es posible; retorna None o algún valor por defecto
    if max_rating == min_rating:
        return None
    else:
        return (player_rating - min_rating) / (max_rating - min_rating)

# Convertir la función Python en una UDF de Spark
udf_normalize_score = udf(normalize_score, FloatType())
udf_normalize_rating = udf(normalize_rating, FloatType())

# Aplicar la UDF para calcular la puntuación normalizada
players_match = players_match.withColumn('normalized_position_score', udf_normalize_score('position_score', 'min_score', 'max_score'))
players_match = players_match.withColumn('normalized_position_rating', udf_normalize_score('player_rating', 'min_rating', 'max_rating'))
players_match.filter(col('player_name') == 'Jack Cork') \
             .orderBy(col('normalized_position_score').desc()) \
             .show(40)

+-------+-------+----------+----------+-----------+---------+--------------------+-------------+---------------+-------+-----+----------+---------------+-------------+----------+-----------+-----+-----------------+------------+-----------+----------------+-------------------+----------------+-----+------------+-----------+--------------+-----------+------------------+----------------+-------------+--------+------------------+---------+---------------+--------------+------------+------------+------------+--------------+---------+---------+----------+----------+-------------------------+--------------------------+
|game_id|team_id| team_name|      date|player_name|player_id|player_position_info|player_rating|good_high_claim|touches|saves|total_pass|formation_place|accurate_pass|aerial_won|aerial_lost|fouls|total_scoring_att|total_tackle|won_contest|penalty_conceded|blocked_scoring_att|man_of_the_match|goals|att_pen_goal|goal_assist|six_yard_block|yellow_card|error_lead_to_goal|post_scori

In [ ]:
players_match.createOrReplaceTempView("players_data")

In [ ]:
# Ejecutando la consulta desde un entorno Python puro en Spark
top_five_players_by_position = spark.sql("""
WITH PlayerAverages AS (
    SELECT
        player_name AS JUGADOR,
        player_position_info AS POSICION,
        AVG(normalized_position_score) AS PROMEDIO_PERFORMANCE_NORMALIZADO
    FROM players_data
    GROUP BY player_name, player_position_info
),
PositionPercentiles AS (
    SELECT
        POSICION,
        percentile_approx(PROMEDIO_PERFORMANCE_NORMALIZADO, 0.50) AS PERCENTIL_50,
        percentile_approx(PROMEDIO_PERFORMANCE_NORMALIZADO, 0.75) AS PERCENTIL_75,
        percentile_approx(PROMEDIO_PERFORMANCE_NORMALIZADO, 1.00) AS PERCENTIL_100
    FROM PlayerAverages
    GROUP BY POSICION
),
PercentileRanks AS (
    SELECT
        PA.JUGADOR,
        PA.POSICION,
        PA.PROMEDIO_PERFORMANCE_NORMALIZADO,
        ABS(PA.PROMEDIO_PERFORMANCE_NORMALIZADO - PP.PERCENTIL_50) AS DISTANCE_TO_50,
        ABS(PA.PROMEDIO_PERFORMANCE_NORMALIZADO - PP.PERCENTIL_75) AS DISTANCE_TO_75,
        ABS(PA.PROMEDIO_PERFORMANCE_NORMALIZADO - PP.PERCENTIL_100) AS DISTANCE_TO_100
    FROM PlayerAverages PA
    JOIN PositionPercentiles PP ON PA.POSICION = PP.POSICION
),
RankedPlayers AS (
    SELECT
        JUGADOR,
        POSICION,
        PROMEDIO_PERFORMANCE_NORMALIZADO,
        DISTANCE_TO_50,
        DISTANCE_TO_75,
        DISTANCE_TO_100,
        DENSE_RANK() OVER (PARTITION BY POSICION ORDER BY DISTANCE_TO_50) AS RANK_50,
        DENSE_RANK() OVER (PARTITION BY POSICION ORDER BY DISTANCE_TO_75) AS RANK_75,
        DENSE_RANK() OVER (PARTITION BY POSICION ORDER BY DISTANCE_TO_100) AS RANK_100
    FROM PercentileRanks
)
SELECT
    RP.JUGADOR,
    RP.POSICION,
    RP.PROMEDIO_PERFORMANCE_NORMALIZADO,
    CASE
        WHEN RP.RANK_50 <= 2 THEN '50th Percentile'
        WHEN RP.RANK_75 <= 2 THEN '75th Percentile'
        WHEN RP.RANK_100 <= 2 THEN '100th Percentile'
        ELSE 'Other'
    END AS PERCENTIL
FROM RankedPlayers RP
WHERE (RP.POSICION = 'DFC' AND (RP.RANK_50 <= 2 OR RP.RANK_75 <= 2 OR RP.RANK_100 <= 2)) OR
      (RP.POSICION = 'MC' AND (RP.RANK_50 <= 3 OR RP.RANK_75 <= 3 OR RP.RANK_100 <= 3)) OR
      (RP.POSICION NOT IN ('DFC', 'MC') AND (RP.RANK_50 = 1 OR RP.RANK_75 = 1 OR RP.RANK_100 = 1))
ORDER BY RP.POSICION, PERCENTIL, RP.PROMEDIO_PERFORMANCE_NORMALIZADO;

""")

# Mostrar los resultados
top_five_players_by_position.show(35)

+-------------------+--------+--------------------------------+----------------+
|            JUGADOR|POSICION|PROMEDIO_PERFORMANCE_NORMALIZADO|       PERCENTIL|
+-------------------+--------+--------------------------------+----------------+
|         Harry Kane|      DC|             0.42125541304011604|100th Percentile|
|     Guido Carrillo|      DC|             0.12183825618454389| 50th Percentile|
|         Chris Wood|      DC|              0.1664068945683539| 75th Percentile|
|    Aymeric Laporte|     DFC|              0.5755952364868588|100th Percentile|
|   Nicolás Otamendi|     DFC|              0.5849264726481017|100th Percentile|
|    Adrian Mariappa|     DFC|             0.30318877526691984| 50th Percentile|
| Christian Kabasele|     DFC|              0.3054846946948341| 50th Percentile|
|      Harry Maguire|     DFC|              0.3567199207058078| 75th Percentile|
|     Chris Smalling|     DFC|              0.3583128117185471| 75th Percentile|
|      Mohamed Salah|      E

In [ ]:
top_five_players_by_position.createOrReplaceTempView("juga_finales")

In [ ]:
# Consulta SQL para unir las vistas en el nombre del jugador y obtener el equipo
query = """
SELECT DISTINCT jf.JUGADOR, jf.POSICION, jf.PROMEDIO_PERFORMANCE_NORMALIZADO, jf.PERCENTIL, pd.team_name AS EQUIPO
FROM juga_finales jf
JOIN players_data pd ON jf.JUGADOR = pd.player_name
"""

# Ejecutar la consulta SQL y almacenar el resultado en un DataFrame
result_df = spark.sql(query)

# Mostrar el resultado
result_df.show(35)


+-------------------+--------+--------------------------------+----------------+--------------------+
|            JUGADOR|POSICION|PROMEDIO_PERFORMANCE_NORMALIZADO|       PERCENTIL|              EQUIPO|
+-------------------+--------+--------------------------------+----------------+--------------------+
|         Harry Kane|      DC|             0.42125541304011604|100th Percentile|   Tottenham Hotspur|
|         Chris Wood|      DC|              0.1664068945683539| 75th Percentile|          Burnley FC|
|     Guido Carrillo|      DC|             0.12183825618454389| 50th Percentile|      Southampton FC|
|   Nicolás Otamendi|     DFC|              0.5849264726481017|100th Percentile|  Manchester City FC|
|    Aymeric Laporte|     DFC|              0.5755952364868588|100th Percentile|  Manchester City FC|
|     Chris Smalling|     DFC|              0.3583128117185471| 75th Percentile|Manchester United FC|
|      Harry Maguire|     DFC|              0.3567199207058078| 75th Percentile|  

In [ ]:
# Asumiendo que result_df es tu DataFrame de Spark
pandas_df = result_df.toPandas()

# Guarda el DataFrame de Pandas como un archivo Excel
excel_file = '/content/FINAL.xlsx'
pandas_df.to_excel(excel_file, index=False)

from google.colab import files
files.download(excel_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>